In [1]:
from api import data_extractor, data_extractor__common
from api import beam__common
from api import fidscs_globals

from apache_beam.io.filesystems import FileSystems
from apache_beam.io.localfilesystem import LocalFileSystem

import google.cloud.storage as gcs
from apache_beam.io.gcp import gcsio

beam_gcp_project = 'sc-fids-capstone'
gcs_bucket = 'sc-fids-capstone-bucket-sc-fids-capstone'
fidscs_globals.WORK_DIR = 'gs://'+gcs_bucket
print(f"fidscs_globals.WORK_DIR: {fidscs_globals.WORK_DIR}")

gcp_auth_key_path = os.environ['GOOGLE_APPLICATION_CREDENTIALS']
fidscs_globals.GCS_CLIENT = gcs.Client.from_service_account_json(gcp_auth_key_path)
print(f"fidscs_globals.GCS_CLIENT: {fidscs_globals.GCS_CLIENT}")

fidscs_globals.GCS_IO = gcsio.GcsIO()   # gcsio.GcsIO(storage_client=fidscs_globals.GCS_CLIENT)
print(f"fidscs_globals.GCS_IO: {fidscs_globals.GCS_IO}")

fidscs_globals.WORK_DIR: gs://sc-fids-capstone-bucket-sc-fids-capstone
fidscs_globals.GCS_CLIENT: <google.cloud.storage.client.Client object at 0x7f992ff781c0>
fidscs_globals.GCS_IO: <apache_beam.io.gcp.gcsio.GcsIO object at 0x7f9926eeefa0>


In [2]:
fidscs_globals.GCS_BUCKET = gcs.Bucket(fidscs_globals.GCS_CLIENT, name=gcs_bucket, user_project=beam_gcp_project)
print(f"fidscs_globals.GCS_BUCKET: {fidscs_globals.GCS_BUCKET}")

all_blobs = list(fidscs_globals.GCS_CLIENT.list_blobs(gcs_bucket))
print(f"\nblobs in gcs bucket ({gcs_bucket}):\n{all_blobs}")

blob_test_folder = fidscs_globals.GCS_BUCKET.blob('test_folder/')
print(f"\ntest_folder: {blob_test_folder}, exists: {blob_test_folder.exists(fidscs_globals.GCS_CLIENT)}")

blob_not_exist_folder = fidscs_globals.GCS_BUCKET.blob('not_exist/')
print(f"not_exist_folder: {blob_not_exist_folder}, exists: {blob_not_exist_folder.exists(fidscs_globals.GCS_CLIENT)}")

blob_video_index_file = fidscs_globals.GCS_BUCKET.blob('test_folder/video_index-20120129.csv')
print(f"video_index-20120129.csv file: {blob_video_index_file}, exists: {blob_video_index_file.exists(fidscs_globals.GCS_CLIENT)}")

test_folder_blobs = list(fidscs_globals.GCS_CLIENT.list_blobs(gcs_bucket, prefix='test_folder/'))
print(f"\nblobs in gcs bucket ({gcs_bucket}) test_folder:\n{test_folder_blobs}")

test_folder_sub_dir_blobs = list(fidscs_globals.GCS_CLIENT.list_blobs(gcs_bucket, prefix='test_folder/sub_dir'))
print(f"\nblobs in gcs bucket ({gcs_bucket}) test_folder_sub_dir_blobs:\n{test_folder_sub_dir_blobs}")

fidscs_globals.GCS_BUCKET: <Bucket: sc-fids-capstone-bucket-sc-fids-capstone>

blobs in gcs bucket (sc-fids-capstone-bucket-sc-fids-capstone):
[<Blob: sc-fids-capstone-bucket-sc-fids-capstone, data/, 1612417860117900>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, data/stitched_video_frames/, 1612401872177603>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, data/tmp/, 1612401871328771>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, data/videos/, 1612401871830298>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, dataflow/staging/sc-fids-capstone-etl.1612406192.684133/apache_beam-2.26.0-cp38-cp38-manylinux1_x86_64.whl, 1612406202565266>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, dataflow/staging/sc-fids-capstone-etl.1612406192.684133/dataflow_python_sdk.tar, 1612406197752629>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, dataflow/staging/sc-fids-capstone-etl.1612406192.684133/pickled_main_session, 1612406202714786>, <Blob: sc-fids-capstone-bucket-sc-fids-capstone, dat

In [3]:
beam__common.make_dirs('gs://'+gcs_bucket+'/data')

In [4]:
l = beam__common.list_dir('gs://'+gcs_bucket+'/data', exclude_subdir=False)
print(f"{l if len(l)>0 else 'empty'}")

l = beam__common.list_dir('gs://'+gcs_bucket+'/test_folder/', exclude_subdir=True)
print(f"{l if len(l)>0 else 'empty'}")

['stitched_video_frames/', 'tmp/', 'videos/']
empty


In [5]:
beam__common.delete_file('gs://'+gcs_bucket+'/test_folder', recursive=True)

True

In [6]:
data_extractor__common.download('http://secrets.rutgers.edu/dai/xml/ncslgr-xml.zip', 'gs://'+gcs_bucket+'/data/ncslgr-corpus-index.csv')

  1%|          | 8192/835568 [00:00<00:13, 61308.72it/s]Downloading http://secrets.rutgers.edu/dai/xml/ncslgr-xml.zip (filesize: 835568 bytes) to gs://sc-fids-capstone-bucket-sc-fids-capstone/data/ncslgr-corpus-index.csv...

100%|██████████| 835568/835568 [00:02<00:00, 326391.84it/s]Successfully downloaded 835568/835568 bytes from URL file http://secrets.rutgers.edu/dai/xml/ncslgr-xml.zip to local file gs://sc-fids-capstone-bucket-sc-fids-capstone/data/ncslgr-corpus-index.csv!

